In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F # All functions that don't have any parameters
import numpy as np
from  boxScore  import boxScore
from sklearn.model_selection import KFold

from torch.utils.data import DataLoader # data management 
# 
from NeuralNetwork import NN
from DatasetPrivate import MyDataset
import numpy as np


import winsound
import wandb

# Preparation of Data

In [9]:
years="2019-20"
# stats="traditional"
stats='advance'
box_score=boxScore(years,stats)

x_train, x_test, y_train, y_test=box_score.separation()
y_train=np.array(y_train)
y_test=np.array(y_test)

input_size=len(x_train.columns)
activation=['relu','sigmoid']
number_neurons=[10,30,50,100,150,200]
# number_neurons=[10,30]
possible_learning_rate=[0.0001,0.001,0.01]
# possible_learning_rate=[0.0001]
num_epochs=500
batch_size=150
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
kf = KFold(n_splits=10, random_state=1, shuffle=True)
train_LL=DataLoader(MyDataset(x_train,y_train),batch_size=batch_size,shuffle=False)
test_LL=DataLoader(MyDataset(x_test,y_test),batch_size=batch_size,shuffle=False)
kf = KFold(n_splits=10, random_state=1, shuffle=True)

# First 

In [11]:
best_config_array=[]
best_history_array=[]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
                
                # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "architecture": "OneHiddenLayer",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
               
                
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp

# winsound.Beep(440,2500)

Best model {'learning_rate': 0.001, 'num_neurons': 10, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2021-22_advance', 'loss': 0.6278985142707825, 'acc': 0.8728333711624146, 'val_loss': 0.6675421595573425, 'val_acc': 0.6140000224113464}
Best model {'learning_rate': 0.01, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2021-22_advance', 'loss': 0.5173345804214478, 'acc': 0.8978333473205566, 'val_loss': 1.2046533823013306, 'val_acc': 0.6079999804496765}
Best model {'learning_rate': 0.001, 'num_neurons': 50, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'architecture': 'OneHiddenLayer', 'dataset': '2021-22_advance', 'loss': 0.584811270236969, 'acc': 0.8770000338554382, 'val_loss': 0.7145747542381287, 'val_acc': 0.6100000739097595}
Best model {'learning_rate': 0.001, 'num_neurons': 100, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'archit

In [12]:
winsound.Beep(440,2500)

In [10]:
def wandbWrite(project_name,best_config_array,best_history_array,name_runs):
    for model_history,config in zip(best_history_array,best_config_array):
        
        

        run = wandb.init(project=project_name, config=config)
        name=name_runs+str(config["num_neurons"])
        run.name=name
        config = wandb.config
        for epoch in range (len(model_history[0])):
            wandb.log({'epochs': epoch,
                'loss': round( model_history[0][epoch].item(),3),
                'acc': round(model_history[1][epoch].item(),3), 
                'val_loss': round(model_history[2][epoch].item(),3),
                'val_acc':round(model_history[3][epoch].item(),3)
                }
                )
        
        run.finish()

In [ ]:
wandb.login()



project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"
wandbWrite(project_name,best_config_array,best_history_array,"onehidden_")


<!-- Best model {'learning_rate': 0.001, 'num_neurons': 10, 'acti_fun': 'relu', 'loss': 0.0193, 'acc': 0.9987, 'val_loss': 1.4627, 'val_acc': 0.6}
Best model {'learning_rate': 0.0001, 'num_neurons': 30, 'acti_fun': 'sigmoid', 'loss': 0.4884, 'acc': 0.9517, 'val_loss': 0.6466, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 50, 'acti_fun': 'sigmoid', 'loss': 0.5163, 'acc': 0.9437, 'val_loss': 0.656, 'val_acc': 0.66}
Best model {'learning_rate': 0.0001, 'num_neurons': 100, 'acti_fun': 'relu', 'loss': 0.0628, 'acc': 0.9951, 'val_loss': 0.775, 'val_acc': 0.64}
Best model {'learning_rate': 0.001, 'num_neurons': 150, 'acti_fun': 'relu', 'loss': 0.0135, 'acc': 0.9998, 'val_loss': 1.4478, 'val_acc': 0.64}
Best model {'learning_rate': 0.0001, 'num_neurons': 200, 'acti_fun': 'relu', 'loss': 0.0487, 'acc': 1.0, 'val_loss': 0.8261, 'val_acc': 0.64} -->

# L2 Regolarization


In [11]:
best_config_array=[]
best_history_array=[]
possible_pL2=[0.0001,0.001,0.01]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
            for pl2 in possible_pL2:
            # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el,pl2=pl2).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "pl2":pl2,
                    "architecture": "OneHiddenLayerL2",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
            
                
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp

# winsound.Beep(440,2500)

Best model {'learning_rate': 0.01, 'num_neurons': 10, 'acti_fun': 'sigmoid', 'batch_size': 150, 'epochs': 500, 'pl2': 0.001, 'architecture': 'OneHiddenLayerL2', 'dataset': '2019-20_advance', 'loss': 0.5272576212882996, 'acc': 0.8816666007041931, 'val_loss': 0.8052139282226562, 'val_acc': 0.7000000476837158}


In [ ]:
winsound.Beep(440,2500)

In [ ]:
wandb.login()

project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"
wandbWrite(project_name,best_config_array,best_history_array,"onehiddenL2_")


# Dropout

In [ ]:
best_config_array=[]
best_history_array=[]
possible_p=[0.5,0.6,0.65]
for num in number_neurons:
    best_config_tmp={'val_acc':0}
    best_history_tmp=[]
    for el in possible_learning_rate:
        for act in activation:
            for p in possible_p:
            # Initialize model
                model = NN(input_size=input_size,size_hidden_level=num,act=act,learning_rate=el,dropout=p).to(device)

                
                
                config={
                    "learning_rate":el,
                    'num_neurons':num,
                    'acti_fun':act,
                    "batch_size": batch_size,
                    "epochs": num_epochs,
                    "drop_rate":p,
                    "architecture": "OneHiddenLayerDropout",
                    "dataset":years+"_"+stats
                }
                
                best_config_tmp,best_history_tmp=model.MyTrain(kf=kf,config=config,best_config_tmp=best_config_tmp,
                best_history_tmp=best_history_tmp,train_LL=train_LL,test_LL=test_LL,device=device)
            
                
                    
    print("Best model",best_config_tmp)
    best_config_array.append(best_config_tmp)
    best_history_array.append(best_history_tmp)
    
# Doing the mean between the 10 fold
for num,array in  enumerate(best_history_array):
    tmp=[ torch.mean(array[0],axis=0),torch.mean(array[1],axis=0),torch.mean(array[2],axis=0),torch.mean(array[3],axis=0)]
  
    best_history_array[num]=tmp



In [ ]:
winsound.Beep(440,2500)

In [ ]:
wandb.login()

project_name="T"+stats[0:3].capitalize()+years[2:].replace("-","")+"runs"

wandbWrite(project_name,best_config_array,best_history_array,"onehiddenDrop_")

# AUTOENCODERS ONE LEVEL

In [ ]:


combination_of_neurons=[[20,10],[15,5]]
for comb in combination_of_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
    
        autoencoder,encoder,decoder = exT.autoencoderOneLevel(input_dimension,comb[0],comb[1])
        loss_fn = tf.keras.losses.BinaryCrossentropy()               
        autoencoder.compile(optimizer="adam",loss=loss_fn,metrics=['accuracy'])
        config={
                "learning_rate":el,
                'num_neurons':comb,
                'acti_fun':"relu"
                
            }
        tik=autoencoder.fit(x_train,x_train,batch_size=200,verbose=False,epochs=50,validation_data=(x_test,x_test))
        config['val_acc']=tik.history['val_accuracy'][-1]
        if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=autoencoder
                    best_config_tmp=config
                 
    print("Best model",best_config_tmp)
    # bestModelArray.append(bestModel)

In [ ]:
model = tf.keras.models.load_model('saved_model_'+stats+'/bestModel')

optimizer = keras.optimizers.SGD(learning_rate=el)
                # Instantiate  loss function.
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=optimizer,loss=loss_fn,metrics=['accuracy'])

# AUTOENCODERS SECOND LEVEL

In [ ]:
combination_of_neurons=[[20,15,5],[15,10,5]]
for comb in combination_of_neurons:
    best_config_tmp={'val_acc':0}
    for el in possible_learning_rate:
    
        autoencoder,encoder,decoder = exT.autoencoderTwoLevels(input_dimension,comb[0],comb[1],comb[2])
        loss_fn = tf.keras.losses.BinaryCrossentropy()               
        autoencoder.compile(optimizer="adam",loss=loss_fn,metrics=['accuracy'])
        config={
                "learning_rate":el,
                'num_neurons':comb,
                'acti_fun':"relu"
                
            }
        tik=autoencoder.fit(x_train,x_train,batch_size=200,verbose=False,epochs=50,validation_data=(x_test,x_test))
        config['val_acc']=tik.history['val_accuracy'][-1]
        if(config['val_acc']>best_config_tmp['val_acc']):
                    bestModel=autoencoder
                    best_config_tmp=config
                 
    print("Best model",best_config_tmp)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_acc=-1
for el in range(2,10):
    clf=RandomForestClassifier(max_depth=el, random_state=42)
    clf.fit(x_train,y_train)
    config=clf.predict(x_test)
    count=0
    for x,y in zip(config,y_test):
        if(x[0]==y[0]):
            count+=1
    acc=count/len(x_test)
    if acc> best_acc:
        best_acc=acc
print(best_acc,el)

In [ ]:
para_y_train=[]
for el1 in y_train:
    tmp=1
    if(el1[0]==1):
        tmp=0
    para_y_train.append(tmp)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
best_acc=-1
for el in range(2,10):
    clf=RandomForestClassifier(n_estimators=100,max_depth=el, random_state=42)
    clf.fit(x_train,para_y_train)
    config=clf.predict(x_test)
    count=0
    for x,y in zip(config,y_test):
        if(x==y[1]):
            count+=1
    acc=count/len(x_test)
    if acc> best_acc:
        best_acc=acc
print(best_acc,el)

# ADA

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

best_acc=-1
clf = AdaBoostClassifier(n_estimators=100, random_state=42)
clf.fit(x_train,para_y_train)
config=clf.predict(x_test)
count=0
for x,y in zip(config,y_test):
    if(x==y[1]):
        count+=1
acc=count/len(x_test)
if acc> best_acc:
    best_acc=acc
print(best_acc)